In [1]:
import sys
print("Python version:", sys.version)

Python version: 3.12.3 | packaged by Anaconda, Inc. | (main, May  6 2024, 19:42:21) [MSC v.1916 64 bit (AMD64)]


In [2]:
pip show llama-cpp-python


Name: llama_cpp_python
Version: 0.2.77
Summary: Python bindings for the llama.cpp library
Home-page: 
Author: 
Author-email: Andrei Betlen <abetlen@gmail.com>
License: MIT
Location: c:\Users\Preetham\miniconda3\envs\new-spec-env\Lib\site-packages
Requires: diskcache, jinja2, numpy, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
from llama_cpp.llama_cpp import _load_shared_library

def is_gpu_available_v3() -> bool:
    lib = _load_shared_library('llama')
    return bool(lib.llama_supports_gpu_offload())

In [4]:
is_gpu_available_v3()

True

In [5]:
import torch

In [6]:
torch.cuda.is_available()

True

In [7]:
from langchain.embeddings import LlamaCppEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
#import pdfplumber
from langchain.text_splitter import CharacterTextSplitter
import os
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from llama_cpp import Llama
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms  import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

In [8]:
#first version
#now we load in the pdfs
document = []
for file in os.listdir("docs"):
    print(file)
    if file.endswith(".pdf"):
        pdf_path = ".\\docs\\"+file
        loader = PyPDFLoader(pdf_path)
        document.extend(loader.load())


ip11pmaxspecs.pdf
iphone13pm.pdf
iphoneSEspecs.pdf


In [6]:
# #second version
# def extract_text_from_pdf(file_path):
#     with pdf.plumber.open(file_path) as pdf:
#         text = ""
#         for page in pdf.pages:
#             text += page.extract_text()
        
#     return text

# def extract_many_files(folder_name):
#     for file in os.listdir(folder_name):
#         if file.endswith(".pdf"):
#             pdf_path = ".\\" + folder_name + "\\"+file
#             text = extract_text_from_pdf(pdf_path)
        

In [9]:
#tried 500, 0
#tried 1000, 200
document_splitta=CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
document_chunks=document_splitta.split_documents(document)
len(document_chunks)

6

In [10]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

c:\Users\Preetham\miniconda3\envs\new-spec-env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\Preetham\miniconda3\envs\new-spec-env\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\Preetham\miniconda3\envs\new-spec-env\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always r

In [11]:
vectordb = Chroma.from_documents(document_chunks, embedding=embeddings, persist_directory='.\\data')
vectordb.persist()

c:\Users\Preetham\miniconda3\envs\new-spec-env\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
c:\Users\Preetham\miniconda3\envs\new-spec-env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [12]:
model_name = "QuantFactory/Meta-Llama-3-8B-GGUF"
model_file = "Meta-Llama-3-8B.Q8_0.gguf"
model_path = ".\\llms\\Meta-Llama-3-8B-Instruct.Q8_0.gguf"

In [11]:
# llm = Llama(
#     model_path=model_path,
#     n_ctx=4096,
#     n_gpu_layers=80,
#     n_batch=521
# )


In [13]:
llm = LlamaCpp(model_path=model_path,temperature=0.3,n_gpu_layers= 80,
        vocab_only = False,
        use_mmap = True,
        use_mlock = False,
        # Context Params                                                                                                                                                                                                                                                          
        seed=  0xFFFFFFFF,
        n_ctx = 4096,
        n_batch = 521,
        n_threads= None,
        rope_freq_base = 0.0,
        rope_freq_scale = 0.0,
        f16_kv = True,
        logits_all = False,
        embedding = False,
        # Sampling Params                                                                                                                                                                                                                                                         
        last_n_tokens_size = 64,
        # LoRA Params                                                                                                                                                                                                                                                             
        lora_base = None,
        lora_scale = 1.0,
        lora_path = None,
        # Backend Params                                                                                                                                                                                                                                                          
        numa= False,
        # Chat Format Params                                                                                                                                                                                                                                                      
        # Misc                                                                                                                                                                                                                                                                    
        verbose = True,
        max_tokens=5000)

c:\Users\Preetham\miniconda3\envs\new-spec-env\Lib\site-packages\langchain_core\utils\utils.py:161: UserWarning: WARNING! embedding is not default parameter.
                embedding was transferred to model_kwargs.
                Please confirm that embedding is what you intended.
  warnings.warn(
c:\Users\Preetham\miniconda3\envs\new-spec-env\Lib\site-packages\langchain_core\utils\utils.py:161: UserWarning: WARNING! lora_scale is not default parameter.
                lora_scale was transferred to model_kwargs.
                Please confirm that lora_scale is what you intended.
  warnings.warn(
c:\Users\Preetham\miniconda3\envs\new-spec-env\Lib\site-packages\langchain_core\utils\utils.py:161: UserWarning: WARNING! numa is not default parameter.
                numa was transferred to model_kwargs.
                Please confirm that numa is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from .\llms\Meta-Llama-3-8B-I

In [13]:
def rag_it(question):
    """Given a question return an answer using the rag pipeline"""
    question = question[0:-1] + "using only the documents given and no prior knowledge"
    docs = vectordb.similarity_search(question)
    rag_pipeline = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",retriever=vectordb.as_retriever())
    print("Here are the docs that were picked up: ")
    for doc in docs:
        print(doc)
    return (rag_pipeline(question))['result']

In [14]:
question = "Given these documents, give me the difference between the iPhone 11 Pro Max and the iPhone 13 Pro Max's features."
docs = vectordb.similarity_search(question)
docs

[Document(page_content='iPhone 13 Pro  Max  Specifications  \nNETWORK  Technology  GSM  / CDMA  / HSPA  / EVDO  / LTE / 5G    \nLAUNCH  Announced  2021, September 14  \nStatus  Release 2021, September 24  \nBODY  Dimensions  160.8 x 78.1 x 7.7 mm (6.33 x 3.07 x 0.30 in)  \nWeight  240 g (8.47 oz)  \nBuild  Glass front (Gorilla Glass), glass back (Gorilla Glass), stainless steel frame  \nSIM  Single SIM (Nano -SIM and/or eSIM) or Dual SIM (Nano -SIM/eSIM, dual stand -by) \n  IP68 dust/water resistant (up to 6m for 30 mins)  \nApple Pay (Visa, MasterCard, AMEX certified)  \nDISPLAY  Type  Super Retina XDR OLED, 120Hz, HDR10, Dolby Vision, 1000 nits (typ), 1200 nits \n(peak)  \nSize  6.7 inches, 109.8 cm2 (~87.4% screen -to-body ratio)  \nResolution  1284 x 2778 pixels, 19.5:9 ratio (~458 ppi density)  \nProtection  Scratch -resistant ceramic glass, oleophobic coating  \n  Wide color gamut  \nTrue-tone \nPLATFORM  OS  iOS 15  \nChipset  Apple A15 Bionic (5 nm)  \nCPU  Hexa -core (2x3.22 G

In [15]:
rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff",retriever=vectordb.as_retriever()
)

In [16]:
answer = rag_pipeline("Given these documents, give me the difference between the iPhone 11 Pro Max and the iPhone 13 Pro Max's features using only these documents and not prior knowledge.")


c:\Users\Preetham\miniconda3\envs\new-spec-env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [ ]:
print(answer['result'])

In [ ]:
question = "What are the dimensions of iPhone 11 Pro Max and iPhone SE?"
answer2 =  rag_it(question)
print(answer2)

In [ ]:
question = "What are the display sizes of the iPhone 11 Pro Max and iPhone SE?"
answer =  rag_it(question)
print(answer)

In [ ]:
question = "What is the display of iPhone 11 Pro Max?"
answer =  rag_it(question)
print(answer)

In [ ]:
question = "What are the display sizes of the iPhone SE and iPhone 13 Pro Max?"
answer =  rag_it(question)
print(answer)

In [ ]:
# memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
# prompt_template = PromptTemplate.from_template('Use these documents to answer questions that I ask about them')

In [ ]:
# llama_model = LLMChain(llm=llm, prompt=prompt_template)
# pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm, retriever=vectordb.as_retriever(search_kwargs={'k':6}),verbose=False,memory=memory)

In [ ]:
# query="What is the main topic of the documents?"
# response=pdf_qa({"question": query})
# print(response["answer"])

In [ ]:
# query="Given these documents, give me the difference between the iPhone 11 Pro max and the iPhone 13 Pro Max features."
# response=pdf_qa({"question": query})
# print(response["answer"])